In [1]:
!nvidia-smi

Wed Feb  7 18:05:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         Off| 00000000:41:00.0 Off |                  N/A |
| 30%   21C    P8               26W / 350W|      0MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip uninstall -y torch
!pip uninstall -y pyg_lib
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
import torch
pytorch_version = f'torch-{torch.__version__}.html'
print(pytorch_version)
!pip install pyg-lib==0.3.1 -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
!pip install torch-geometric
!pip install osmnx
import pyg_lib
print("pyg_lib version: ", pyg_lib.__version__)
!python -c "from IPython.display import clear_output; clear_output(); print('Installed requirements')"
# %cd  /home/jovyan/CPrompt

Found existing installation: torch 2.0.0+cu117
Uninstalling torch-2.0.0+cu117:
  Successfully uninstalled torch-2.0.0+cu117
Found existing installation: torchvision 0.15.1+cu117
Uninstalling torchvision-0.15.1+cu117:
  Successfully uninstalled torchvision-0.15.1+cu117
Found existing installation: torchaudio 2.0.1+cu117
Uninstalling torchaudio-2.0.1+cu117:
  Successfully uninstalled torchaudio-2.0.1+cu117
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 3.3 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.4 MB/s eta 0:00:0000:0100:01
  Using cached https://download.pytorch.org/whl/triton-2.1.0-0-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninst

In [2]:
%cd  /home/jovyan/CPrompt
import torch, random, os
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List
from torch_geometric.data import Data, Batch
from torchvision.transforms.functional import normalize
from torch_geometric.utils import augmentation, to_dense_adj
from torch_geometric.nn import GAT, GATConv, GCNConv
from torch_geometric.datasets import QM9, TUDataset
from torch_geometric.loader import DataLoader, DenseDataLoader
from torch_geometric.nn import global_mean_pool
from utils import *
from model import GCN, LinkPredictionPrompt
from copy import deepcopy
from torch_geometric.data import Data
from torch_geometric.datasets import Amazon
from torch_geometric.loader import NeighborLoader
from torch.utils.data import Dataset

/home/jovyan/CPrompt


In [ ]:
t_loader = NeighborLoader(
    dataset[0],
    input_nodes=torch.arange(dataset.x.size(0)),
    num_neighbors=[-1]*2,
    batch_size=1,
    replace=False,
    shuffle=False,
)
for i, batch in enumerate(t_loader):
    print(i, batch.input_id)
# t_batch = next(iter(t_loader))
# print(dataset[0].y[0], dataset[0].y[t_batch.n_id])

In [40]:
t_batch.edge_index[:, :10]

tensor([[1, 2, 3, 4, 0, 5, 6, 7, 8, 9],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])

In [41]:
t_batch.n_id[:10]

tensor([   0,  507, 6551, 8210, 9745,   31,   40,  182,  430,  468])

In [37]:
t_batch.input_id

tensor([0])

In [42]:
t_batch.e_id[:10]

tensor([ 17728, 236356, 297090, 350263,      0,    934,   1367,   6694,  14827,
         16368])

In [43]:
dataset[0].edge_index[:, t_batch.e_id[:10]]

tensor([[ 507, 6551, 8210, 9745,    0,   31,   40,  182,  430,  468],
        [   0,    0,    0,    0,  507,  507,  507,  507,  507,  507]])

In [53]:
print(dataset[0].y.size())
print(len(node_ds.all_nids))
print(len(t_loader))

torch.Size([13752])
13471
13752


In [54]:
class NodeClassification(Dataset):
    def __init__(self, dataset, n_hopes=2, train_per=0.85, test_per=0.15):
        self._data = dataset[0]
        if train_per + test_per != 1.0:
            valid_per = 1 - (train_per + test_per)
        else:
            valid_per = 0.0
        print("Toral num nodes: ", dataset.x.size(0))
        loader = NeighborLoader(
            self._data,
            input_nodes=torch.arange(dataset.x.size(0)),
            num_neighbors=[-1]*n_hopes,
            batch_size=1,
            replace=False,
            shuffle=False,
        )
        print("Neighborhood Loader Created")
        self.all_edges = []
        self.all_nids = []
        self.r_nodes = []
        for i, batch in enumerate(loader):
            if batch.n_id.size(0) > 1:
                print(f"Batch num: {i}/{len(loader)}", end='\r', flush=True)
                self.r_nodes.append(batch.input_id)
                self.all_nids.append(batch.n_id)
                self.all_edges.append(batch.edge_index)
        self.r_nodes = torch.tensor(self.r_nodes)
        self.n_nodes = len(self.all_nids)
        self.n_feats = dataset.x.size(1)
        self.n_classes = dataset.num_classes
        self.train_idx = int(self.n_nodes * train_per)
        self.n_train = self.train_idx
        self.n_valid = int(self.n_nodes * valid_per)
        self.test_idx = self.train_idx + int(self.n_nodes * valid_per)
        self.n_test = self.n_nodes - self.test_idx
        self.labels = self._data.y

    def _shuffle(self,):
        perm = torch.randperm(self.train_idx)
        self.all_nids[:self.train_idx] = [self.all_nids[i] for i in perm]
        self.all_edges[:self.train_idx] = [self.all_edges[i] for i in perm]
        self.r_nodes[:self.train_idx] = self.r_nodes[perm]

    def __len__(self):
        return self.n_nodes

    def __getitem__(self, indices):
        if isinstance(indices, int):
            indices = [indices]
        elif isinstance(indices, slice):
            indices = [i for i in range(*indices.indices(len(self)))]
        else:
            raise TypeError("Invalid index type. Must be int, slice, or list.")
        all_graphs = []
        for idx in indices:
            x = self._data.x[self.all_nids[idx]]
            y = self._data.y[self.r_nodes[idx]]
            edges = self.all_edges[idx]
            all_graphs.append(Data(x=x, edge_index=edges, y=y))
        return all_graphs, self._data.y[self.r_nodes[indices]]

In [59]:
dataset = Amazon(
    root='data/Amazon',
    name='Computers'
    )
node_ds = NodeClassification(dataset, n_hopes=2, train_per=0.85, test_per=0.15)
node_ds._shuffle()

Toral num nodes:  13752
Neighborhood Loader Created


In [64]:
!cp 

data		 main.py   old_main.py	 old_prompt.py	pretrain.py  README.md
installation.sh  model.py  old_model.py  old_utils.py	__pycache__  utils.py


In [60]:
def dense_to_sparse(spmat):
    indices = spmat.nonzero(as_tuple=False)
    spmat = torch.sparse_coo_tensor(
        indices.T,
        spmat[indices[:, 0], indices[:, 1]],
        size=spmat.size(),
        requires_grad=True
        )
    return spmat

def batch_to_xadj_list(g_batch, device):
    x_adj_list = []
    for i, g in enumerate(g_batch):
        g = g.to(device)
        adj_dense = to_dense_adj(g.edge_index, max_num_nodes=g.x.size(0)).squeeze()
        deg_mat = adj_dense.sum(dim=1)
        deg_mat_inv = torch.max(deg_mat, torch.ones_like(deg_mat)*1e-6).pow(-1)
        deg_mat_inv = deg_mat_inv.diag()
        adj_dense = deg_mat_inv @ adj_dense
        adj_dense.fill_diagonal_(1.)
        adj_sparse = dense_to_sparse(adj_dense)
        x_adj_list.append((g.x, adj_sparse))
    return x_adj_list

def test(model, dataset, batch_size, device, epoch=None, visualize=False, colors=None):
    model.eval()
    test_loss, correct = 0, 0
    for i in range(dataset.test_idx, dataset.n_nodes, batch_size):
        print(f"Test batch: {i}/{node_ds.test_idx}", end='\r')
        test_batch, test_labels = dataset[i:min(i+batch_size, dataset.n_nodes)]
        x_adj_list = batch_to_xadj_list(test_batch, device)
        out, embeds = model(x_adj_list)
        if visualize:
            visualize_and_save_tsne(
                embeds.cpu().detach().numpy(),
                colors[test_labels.cpu()] if colors is not None else None,
                epoch if epoch is not None else 0)
        test_loss += F.cross_entropy(out, test_labels.to(device), reduction="sum")
        out = F.softmax(out, dim=1)
        pred = out.max(dim=1)[1]
        correct += int((pred == test_labels.to(device)).sum())
    test_loss /= dataset.n_test
    test_acc = correct / dataset.n_test
    return test_loss, test_acc

In [63]:
h_dim = 64
n_layers = 2
batch_size = 64
visualize = False
if visualize:
    colors = np.array([
        "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) \
        for i in range(dataset.y.unique().size(0))])
else:
    colors = None

model = GCN(node_ds.n_feats, h_dim, nclass=node_ds.n_classes, dropout=0.2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# for d in train_loader:
#     d.to(device)
obj_fun = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-2)
scheduler = StepLR(optimizer, step_size=100, gamma=0.5)

n_epochs = 300
for epoch in range(n_epochs):
    model.train()
    node_ds._shuffle()
    for i in range(0, node_ds.train_idx, batch_size):
    # for i in range(0, 1000, batch_size):
        print(f"Train batch: {i}/{node_ds.train_idx}", end='\r')
        train_batch, train_labels = node_ds[i:min(i+batch_size, node_ds.train_idx)]
        x_adj_list = batch_to_xadj_list(train_batch, device)
        optimizer.zero_grad()
        emb_out, _ = model(x_adj_list)
        loss = obj_fun(emb_out, train_labels.to(device))
        loss.backward()
        optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    with torch.no_grad():
        test_loss, test_acc = test(
            model, node_ds, 32, device, epoch, visualize, colors)
        print(f'Epoch: {epoch}/{n_epochs}, Train Loss: {loss:.4f}, Main Loss: {test_loss:.4f}, Main ACC: {test_acc:.3f}')

save_model = True
if save_model:
    model_dir = './pretrained'
    os.makedirs(model_dir, exist_ok=True)
    name = datetime.today().strftime('%Y_%m_%d_%H_%M')
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, os.path.join(model_dir, f'model_{name}.pt')
    )

Epoch: 0/300, Train Loss: 0.9923, Main Loss: 0.7932, Main ACC: 0.741
Epoch: 1/300, Train Loss: 0.6196, Main Loss: 0.6549, Main ACC: 0.787
Epoch: 2/300, Train Loss: 0.6400, Main Loss: 0.7300, Main ACC: 0.770
Epoch: 3/300, Train Loss: 0.7597, Main Loss: 0.6820, Main ACC: 0.773
Epoch: 4/300, Train Loss: 0.6139, Main Loss: 0.5821, Main ACC: 0.809
Epoch: 5/300, Train Loss: 0.3591, Main Loss: 0.5804, Main ACC: 0.812
Epoch: 6/300, Train Loss: 0.7764, Main Loss: 0.6175, Main ACC: 0.804


KeyboardInterrupt: 